In [22]:
#import custom module with defined functions
from kinase_module import *

In [30]:
# imports
from __future__ import print_function
import warnings; warnings.simplefilter('ignore')
import sys
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.linear_model import LinearRegression as lr
from sklearn.ensemble import GradientBoostingRegressor as gb
from sklearn.svm import SVC
from sklearn.linear_model import ElasticNet as en
from sklearn import preprocessing
from sklearn.linear_model import Lasso as las
from sklearn.linear_model import BayesianRidge as br
from sklearn.preprocessing import StandardScaler as sc
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn import linear_model 
from sklearn.model_selection import cross_val_score,cross_val_predict

In [31]:
#Data frame
#df = pd.read_csv('~/Desktop/data/kinase_files/KinaseP00533.csv')
#df.head()
df = pd.read_csv('~/Desktop/data/kinase_files/KinaseO00329.csv')
df.head()


,Unnamed: 0,target_id,pAc,smiles,SlogP,SMR,LabuteASA,TPSA,AMW,ExactMW,...,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41,MQN42
0,0,O00329,1.4,CS(=O)(=O)C1=CN=CC(=C1)C2=CC3=NC(=NN3C=C2)N,0.7770,73.7402,114.564286,103.24,289.320,289.063346,...,0,0,1,2,0,0,0,0,2,1
1,1,O00329,1.5,C1=CN2C(=NC(=N2)N)C=C1C3=CC(=CN=C3)S(=O)(=O)N,0.0209,72.0036,113.674613,129.26,290.308,290.058595,...,0,0,1,2,0,0,0,0,2,1
2,2,O00329,1.5,CC(C)NS(=O)(=O)C1=CN=CC(=C1)C2=CC3=NC(=NN3C=C2)N,1.0602,86.0879,132.979410,115.27,332.389,332.105545,...,0,0,1,2,0,0,0,0,2,1
3,3,O00329,1.5,CC1(CCN1S(=O)(=O)C2=CN=CC(=C2)C3=CC4=NC(=NN4C=...,1.5465,93.2182,144.913435,106.48,358.427,358.121195,...,0,1,1,2,0,0,0,0,2,1
4,4,O00329,1.5,CC(C)(C(F)(F)F)NS(=O)(=O)C1=CN=CC(=C1)C2=CC3=N...,1.9927,91.0859,151.840959,115.27,400.386,400.092929,...,0,0,1,2,0,0,0,0,2,1


In [32]:
#Check the shape of the dataframe
df.shape

(1381, 120)

In [34]:
#Create a drop highly correlated columns function 
def drop_corr(df):
    '''Function drops highly correlated columns above a 0.95 threshold and outputs a new dataframe.

    Input:
        df: pandas DataFrame
        '''
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column]>0.95)]
    df_new = df.drop(df[to_drop],axis = 1)
    return(df_new)
df1 = drop_corr(df)
df1.head()

,Unnamed: 0,target_id,smiles,SlogP,SMR,TPSA,NumLipinskiHBA,NumLipinskiHBD,NumRotatableBonds,NumHBD,...,MQN32,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41
0,0,O00329,CS(=O)(=O)C1=CN=CC(=C1)C2=CC3=NC(=NN3C=C2)N,0.7770,73.7402,103.24,7,2,2,1,...,0,0,0,1,2,0,0,0,0,2
1,1,O00329,C1=CN2C(=NC(=N2)N)C=C1C3=CC(=CN=C3)S(=O)(=O)N,0.0209,72.0036,129.26,8,4,2,2,...,0,0,0,1,2,0,0,0,0,2
2,2,O00329,CC(C)NS(=O)(=O)C1=CN=CC(=C1)C2=CC3=NC(=NN3C=C2)N,1.0602,86.0879,115.27,8,3,4,2,...,0,0,0,1,2,0,0,0,0,2
3,3,O00329,CC1(CCN1S(=O)(=O)C2=CN=CC(=C2)C3=CC4=NC(=NN4C=...,1.5465,93.2182,106.48,8,2,3,1,...,1,0,1,1,2,0,0,0,0,2
4,4,O00329,CC(C)(C(F)(F)F)NS(=O)(=O)C1=CN=CC(=C1)C2=CC3=N...,1.9927,91.0859,115.27,8,3,4,2,...,0,0,0,1,2,0,0,0,0,2


In [35]:
#Print shape of the dataframe after dropping highly correlated values 
df1.shape

(1381, 93)

In [36]:
# Get the feature vector
#X = df.drop('smiles', 'mol', 'pAc', axis=1)
X = df1.drop(columns=['Unnamed: 0','target_id','smiles'],axis=1).values
# Get the target vector
y = df["pAc"].values 

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)

In [8]:
#Standardize the data
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.fit_transform(X_test)

MODEL EVALUATION

In [9]:
def model_eval(model):
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    model_score = model.score(X_test,y_test)
    model_mse =  mse(y_test,pred)
    model_r2 =r2(y_test,pred)
    return(model_score, model_mse, model_r2) 

In [10]:
#Linear Regression
model_eval(lr())

(0.4236641773823187, 1.1686617103854677, 0.42366417738231865)

In [11]:
#Random Forest 
model_eval(rf())

(0.6555682564311054, 0.6984195233987598, 0.6555682564311054)

In [18]:
from sklearn import svm
scores = cross_val_score(classifier, features, targets, cv=7)
scores = cross_val_score(svm, X, y, cv=7)

AttributeError: module 'sklearn.svm' has no attribute 'fit'

###### DISCUSSION 
#1.When the training set is standardized, model score = 54% 
#2.When the training set is not standardized, the model score = 65% 
#3.Applying PCA made the model worse since the highly correlated variables were already removed

###### GRADIENTBOOSTINGREGRESSOR

In [ ]:
model_eval(gb())

##### ElasticNet Model

In [ ]:
model_eval(en())

##### BayesianRidge Model

In [ ]:
model_eval(br())

###### DEEP LEARNING

In [28]:
?cross_val_score